In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
cd gdrive/My Drive/Voice_Cloning

/content/gdrive/My Drive/Voice_Cloning


In [3]:
cd StarGAN-Voice-Conversion/

/content/gdrive/My Drive/Voice_Cloning/StarGAN-Voice-Conversion


In [5]:
!python3 prepocess.py

python3: can't open file 'prepocess.py': [Errno 2] No such file or directory


In [14]:
import librosa
import numpy as np
import os, sys
import argparse
import pyworld
from multiprocessing import cpu_count
from concurrent.futures import ProcessPoolExecutor
from functools import partial
from utils import *
from tqdm import tqdm
from collections import defaultdict
from collections import namedtuple
from sklearn.model_selection import train_test_split
import glob
from os.path import join, basename
import subprocess

def resample(spk, origin_wavpath, target_wavpath):
    wavfiles = [i for i in os.listdir(join(origin_wavpath, spk)) if i.endswith(".wav")]
    for wav in wavfiles:
        folder_to = join(target_wavpath, spk)
        os.makedirs(folder_to, exist_ok=True)
        wav_to = join(folder_to, wav)
        wav_from = join(origin_wavpath, spk, wav)
        subprocess.call(['sox', wav_from, "-r", "16000", wav_to])
    return 0

def resample_to_16k(origin_wavpath, target_wavpath, num_workers=1):
    os.makedirs(target_wavpath, exist_ok=True)
    spk_folders = os.listdir(origin_wavpath)
    print(f"> Using {num_workers} workers!")
    executor = ProcessPoolExecutor(max_workers=num_workers)
    futures = []
    for spk in spk_folders:
        futures.append(executor.submit(partial(resample, spk, origin_wavpath, target_wavpath)))
    result_list = [future.result() for future in tqdm(futures)]
    print(result_list)

def split_data(paths):
    indices = np.arange(len(paths))
    test_size = 0.1
    train_indices, test_indices = train_test_split(indices, test_size=test_size, random_state=1234)
    train_paths = list(np.array(paths)[train_indices])
    test_paths = list(np.array(paths)[test_indices])
    return train_paths, test_paths

def get_spk_world_feats(spk_fold_path, mc_dir_train, mc_dir_test, sample_rate=16000):
    paths = glob.glob(join(spk_fold_path, '*.wav'))
    spk_name = basename(spk_fold_path)
    train_paths, test_paths = split_data(paths)
    f0s = []
    coded_sps = []
    for wav_file in train_paths:
        f0, _, _, _, coded_sp = world_encode_wav(wav_file, fs=sample_rate)
        f0s.append(f0)
        coded_sps.append(coded_sp)
    log_f0s_mean, log_f0s_std = logf0_statistics(f0s)
    coded_sps_mean, coded_sps_std = coded_sp_statistics(coded_sps)
    np.savez(join(mc_dir_train, spk_name+'_stats.npz'), 
            log_f0s_mean=log_f0s_mean,
            log_f0s_std=log_f0s_std,
            coded_sps_mean=coded_sps_mean,
            coded_sps_std=coded_sps_std)
    
    for wav_file in tqdm(train_paths):
        wav_nam = basename(wav_file)
        f0, timeaxis, sp, ap, coded_sp = world_encode_wav(wav_file, fs=sample_rate)
        normed_coded_sp = normalize_coded_sp(coded_sp, coded_sps_mean, coded_sps_std)
        np.save(join(mc_dir_train, wav_nam.replace('.wav', '.npy')), normed_coded_sp, allow_pickle=False)
    
    for wav_file in tqdm(test_paths):
        wav_nam = basename(wav_file)
        f0, timeaxis, sp, ap, coded_sp = world_encode_wav(wav_file, fs=sample_rate)
        normed_coded_sp = normalize_coded_sp(coded_sp, coded_sps_mean, coded_sps_std)
        np.save(join(mc_dir_test, wav_nam.replace('.wav', '.npy')), normed_coded_sp, allow_pickle=False)
    return 0




sample_rate = 16000
origin_wavpath = "data/VCTK-Corpus/wav48"
target_wavpath = "data/VCTK-Corpus/wav16"
mc_dir_train = 'data/mc/train'
mc_dir_test = 'data/mc/test'
num_workers = cpu_count()


# # The original wav in VCTK is 48K, first we want to resample to 16K
# resample_to_16k(origin_wavpath, target_wavpath, num_workers=num_workers)

# WE only use 10 speakers listed below for this experiment.
speaker_used = ['225', '226', '227']
speaker_used = ['p'+i for i in speaker_used]

## Next we are to extract the acoustic features (MCEPs, lf0) and compute the corresponding stats (means, stds). 
# Make dirs to contain the MCEPs
os.makedirs(mc_dir_train, exist_ok=True)
os.makedirs(mc_dir_test, exist_ok=True)

num_workers = len(speaker_used) #cpu_count()
print("number of workers: ", num_workers)
executor = ProcessPoolExecutor(max_workers=num_workers)

work_dir = target_wavpath
# spk_folders = os.listdir(work_dir)
# print("processing {} speaker folders".format(len(spk_folders)))
# print(spk_folders)

futures = []
for spk in speaker_used:
  spk_path = os.path.join(work_dir, spk)
  futures.append(executor.submit(partial(get_spk_world_feats, spk_path, mc_dir_train, mc_dir_test, sample_rate)))
result_list = [future.result() for future in tqdm(futures)]
print(result_list)
sys.exit(0)



number of workers:  3





  0%|          | 0/3 [00:00<?, ?it/s]

ValueError: ignored

In [9]:
!ls

converted_samples  data_loader.py  model.py	  README.md
convert.py	   logger.py	   preprocess.py  solver.py
data		   main.py	   __pycache__	  utils.py


In [7]:
!pip3 install pyworld 

     |████████████████████████████████| 81kB 2.2MB/s 
  Created wheel for pyworld: filename=pyworld-0.2.10-cp36-cp36m-linux_x86_64.whl size=610446 sha256=5fc832e0ec539440029678079b66b6235a04813c972bbb27af7bf7ad54ed5ae3
  Stored in directory: /root/.cache/pip/wheels/7d/11/e0/b197a0bafe9a34ade04f4b85a43fd8754b853f4a59e24a9022
Successfully built pyworld
